In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [3]:
import time
from torch.utils.tensorboard import SummaryWriter

In [4]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [5]:
import pandas as pd
import json

In [6]:
from torch.utils.data import DataLoader

In [21]:
from IPython.display import clear_output
from IPython.display import display

In [15]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.fc1(t.reshape(-1, 12 * 4 * 4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [16]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [17]:
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = 0
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
        
    def begin_run(self, run ,network, loader):
        self.run_start_time = time.time()
            
        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        self.tb.add_image('image', grid)
        self.tb.add_graph(self.network, images)
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0
        
    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
    
    def end_epoch(self):
        
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)
        
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
        
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        
        for k, v in self.run_params._asdict().items():
            results[k] = v
            
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns') 
        
        clear_output(wait=True)
        display(df)
        
    def track_loss(self, loss):
        self.epoch_loss += loss.item() * self.loader.batch_size 
    
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)   
        
    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self, fileName):
        
        pd.DataFrame.from_dict(
            self.run_data,
            orient="columns"
        ).to_csv(f'{fileName}.csv')
        
        with open(f'{fileName}.json', 'w', encoding='utf-8')as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)
               
        

In [18]:
class RunBuilder():
    @staticmethod   # 无需创建实例，可通过指定类进行调用的方法
    def get_runs(params):
        Run = namedtuple('Run', params.keys())
        runs = []
        
        for v in product(*params.values()):
            runs.append(Run(*v))
            
        return runs

In [19]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"    # 不加这句下面的跑不出来

In [22]:
params = OrderedDict(
    lr = [.01],
    batch_size = [1000, 2000]
)

m = RunManager()
for run in RunBuilder.get_runs(params):
    
    network = Network()
    loader = DataLoader(train_set, batch_size=run.batch_size)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            
            images = batch[0]
            labels = batch[1]
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds, labels)
            
        m.end_epoch()
    m.end_run()
    
m.save('results')
        

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size
0,1,1,0.944786,0.629233,8.137032,11.824112,0.01,1000
1,1,2,0.535112,0.788867,7.772925,19.678480,0.01,1000
2,1,3,0.455417,0.830617,8.482848,28.228637,0.01,1000
3,1,4,0.399184,0.852867,8.230281,36.518135,0.01,1000
4,1,5,0.361611,0.865767,8.035125,44.619977,0.01,1000
5,2,1,1.206154,0.534317,7.709131,8.709025,0.01,2000
6,2,2,0.635088,0.748967,7.838985,16.606147,0.01,2000
7,2,3,0.528095,0.794667,8.185973,24.843560,0.01,2000
8,2,4,0.465943,0.827350,8.389606,33.299807,0.01,2000
9,2,5,0.422847,0.846450,8.115608,41.487226,0.01,2000
